In [11]:
from wilcoxon.spiderman import *
from wilcoxon import sheets, utils
import re
import pandas as pd
from tqdm import tqdm
import numpy as np
from math import sin, cos, sqrt, atan2, radians

In [149]:
postcode = sheets.getGeo("Postcode").drop_duplicates(subset = "Postcode").set_index("Postcode").to_dict(orient = "index")

In [3]:
site = website("https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore#:~:text=This%20is%20a%20list%20of,171%20malls%20on%20this%20list.")

In [187]:
# listsOfLinks = site.html("ul")[1:8]
listsOfLinks = [x.find("ul") for x in site.html("td")[2:6]]
links = []

for l in listsOfLinks:
    linkElements = l("li")
    for linkElement in linkElements:
        try:
            links.append((linkElement.find("a")["title"], "https://en.wikipedia.org/" + linkElement.find("a")["href"]))
        except:
            links.append((linkElement.text, ""))

In [188]:
L = []
for link in tqdm(links):
    subsite = ""
    myDict = {"name": link[0]
             ,"coords": None
             ,"latitude": None
             ,"longitude": None
             ,"address": None
             ,"postcode": None
             ,"floors": None
             ,"stores": None
             ,"opening date": None
             ,"opening year": None
             ,"area": None
             ,"link": link[1]}
    
    try:
        subsite = website(link[1]).html
        for a in subsite("a"):
            a.replaceWith(a.text)
    except: None
    try: myDict["address"] = subsite.find(class_ = "infobox-label", text = re.compile("ddress")).find_next("td").text
    except: None
    try: myDict["postcode"] = int(re.findall("\d{6}", myDict["address"])[0])
    except: None
    try: myDict["floors"] = subsite.find(class_ = "infobox-label", text = re.compile("loors")).find_next("td").text
    except: None
    try: myDict["stores"] = subsite.find(class_ = "infobox-label", text = re.compile("tores")).find_next("td").text
    except: None
    try:
        myDict["coords"] = re.findall('"coordinates":\[(\d+\.\d+),(\d+\.\d+)\]', str(subsite.find("script")))[0]
        myDict["latitude"] = myDict["coords"][0]
        myDict["longitude"] = myDict["coords"][1]
        myDict["coords"] = ", ".join(myDict["coords"])
    except:
        try:
            myDict["latitude"] = postcode[myDict["postcode"]]["Lat"]
            myDict["longitude"] = postcode[myDict["postcode"]]["Long"]
        except: None
    try: 
        myDict["opening date"] = subsite.find(class_ = "infobox-label", text = re.compile("pening")).find_next("td").text
        myDict["opening year"] = re.findall("\d{4}", myDict["opening date"])[0]
    except: None
    try: myDict["area"] = subsite.find(class_ = "infobox-label", text = re.compile("rea")).find_next("td").text
    except: None

    L.append(myDict)
malls2 = pd.DataFrame(L)


100%|██████████| 156/156 [01:35<00:00,  1.64it/s]


In [78]:
malls = sheets.getGeo("Malls Alternative")
mrt = sheets.getGeo("MRT")
bus = sheets.getGeo("Bus")

In [22]:
def dist(x1,y1,x2,y2):
    R = 6373.0

    lat1 = radians(x1)
    lon1 = radians(y1)
    lat2 = radians(x2)
    lon2 = radians(y2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

In [3]:
def postcodeCoords(location):
    return re.findall("\d+\.\d+", website(f"https://www.google.com/search?q={location}+coordinates").html.find(class_="BNeawe iBp4i AP7Wnd").string)

In [79]:
mrtm = mrt.fillna(0)[mrt.Type == "Station"].drop_duplicates("Name")[["Lat","Long","Origin","Destination"]].values.tolist()
busm = bus.fillna(0).drop_duplicates("BusStopCode")[["Latitude","Longitude","In","Out"]].values.tolist()
mallsm = malls[["Latitude","Longitude"]].values.tolist()

In [80]:
origins = []
destinations = []
for latm, longm in tqdm(mallsm):
    origin = 0
    destination = 0
    for lat, long, o, d in mrtm:
        if dist(latm, longm, lat, long) < 0.5:
            origin += o
            destination += d
    for lat, long, o, d in busm:
        if dist(latm, longm, lat, long) < 0.5:
            origin += o
            destination += d
    origins.append(origin)
    destinations.append(destination)
        

100%|██████████| 120/120 [00:01<00:00, 102.57it/s]


In [81]:
pd.DataFrame([origins, destinations]).T.to_clipboard(index=False, header=False)

In [71]:
bus[bus.fillna("").Description.str.contains("Boon Lay")].drop_duplicates("BusStopCode")

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,RoadName,Description,Latitude,Longitude,Alt,In,Out
10171,246,SBST,1,40,21491,14.3,644,6,632,3,627,3,Quality Rd,Bef Boon Lay Way,1.326864,103.710827,14.456647,4367,993
10456,105,SBST,2,8,28039,3.5,558,2357,557,13,555,2356,Boon Lay Way,Aft Boon Lay Sub Stn,1.328075,103.751408,14.650918,13494,23224
12833,NR5,SMRT,1,76,22571,29.3,-,-,46,316,-,-,Boon Lay Way,Opp Boon Lay Stn,1.337804,103.706891,12.569916,23950,16
13204,154,SBST,1,1,22009,0.0,530,20,530,20,600,20,Jurong West Ctrl 3,Boon Lay Int,1.339323,103.705457,13.248370,1604553,1385204
14706,240,SBST,1,32,21449,12.5,536,116,532,113,546,115,Boon Lay Pl,Boon Lay Shop Ctr,1.346398,103.713061,16.598611,75712,123967
15161,240,SBST,1,33,21439,12.8,538,117,533,114,548,116,Boon Lay Pl,Boon Lay CC,1.348423,103.711659,17.294440,17696,18268
